# 8510 HW6 Part1 - David Tung

In [331]:
import pandas as pd
import numpy as np

# Load the data from file
file_path = './spam2.csv'
spam_data = pd.read_csv(file_path)
np_spam_data = np.array(spam_data)

In [333]:
# Split data into train (top 70%) and test (other 30%) 
split_index = int(0.7 * len(spam_data))
train_data = np_spam_data[:split_index]
test_data = np_spam_data[split_index:]

In [338]:
class_count = np.array([sum(train_data[:,0]==0), sum(train_data[:,0]==1)])
prior_prob = class_count/train_data.shape[0]

In [348]:
class NaiveBayesUtils:
    
    def __init__(self):
        """
        Init function
        """
        self.t_class_f = None # target frequency
        self.feature_f = None
        rows = 0
        columes = 0
        results = []
        self.t_class_p = None # probability
        self.feature_p = None

    def probabilities(self, feature):
        """
        Calculate the probabilities
        """
        n_rows, n_columes = feature.shape
        self.t_class_p = self.t_class_f/n_rows
        for i in range(n_columes):
            num_unique_feature_val = len(np.unique(feature[:, i]))
            for f in range(num_unique_feature_val):
                ##
                ## itrate classes 0,1 
                for cls in [0,1]:
                    self.feature_p[i, f, cls] = \
                     (self.feature_f[i, f, cls])/(self.t_class_f[cls])

    def fit_model(self, feature, y):
        """ 
        Train the NB model 
        """
        n_rows, n_features = feature.shape
        self.t_class_f = np.zeros(2)
        ##
        ## frequncy array of [num of features, num of feature vals, num of target class]
        ##
        self.feature_f = \
            np.zeros((n_features, len(np.unique(feature)), 2))
        self.t_class_p = np.zeros(2)

        ##
        ## probablity array of [num of features, num of feature vals, num of target class]
        ##
        self.feature_p = \
            np.zeros((n_features, len(np.unique(feature)), 2))
        for cls in [0,1]: # iterate classes
            feature_c = feature[y == cls]
            self.t_class_f[cls] = feature_c.shape[0]
            for fi in range(n_features): # iterate feature index
                feature_values = np.unique(feature[:, fi])
                for fv in feature_values: # iterate feature val
                    self.feature_f[fi, fv, cls] = np.sum(feature_c[:, fi] == fv)
                ##print(self.t_class_p)
        self.probabilities(feature)

    def predict_results(self, features):
        """
        Predict result based on test features
        """
        pred = []
        for sample in features:
            posteriors = []
            ##
            ## ite target class 0,1
            for cls in [0,1]:
                posterior = self.t_class_p[cls]
                for idx, val in enumerate(sample):
                    posterior = \
                        posterior * self.feature_p[idx, val, cls]
                posteriors.append(posterior)
            pred.append([0,1][np.argmax(posteriors)])
            # print(pred)
        return np.array(pred)

In [349]:
naiveBayesUtils = NaiveBayesUtils()
features = train_data[:, 1:]  # Features
target = train_data[:, 0]   # Labels
naiveBayesUtils.fit_model(features, target)

testing_data_features = test_data[:, 1:]  # Features from testing
y_pred = naiveBayesUtils.predict_results(testing_data_features)

actual = np.array([test_data[:, 0]]).T[:,0]
pred = np.array([y_pred]).T[:,0]

TP = sum((actual==1) & (pred ==1))
TN = sum((actual==0) & (pred ==0))
FN = sum((actual==1) & (pred ==0))
FP = sum((actual==0) & (pred ==1))

cm = np.array([[TP, FN],
               [FP, TN]])

# Results

In [350]:
Accuracy = (TP+TN)/(TP+FN+FP+TN)
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
F1 = 2*Precision*Recall/(Precision+Recall)
print(f"Class Counts [0, 1]:\n {class_count}\n")
print(f"Prior Probabilities [0, 1]:\n {prior_prob}\n")
print(f"Confusion Matrix[[TP,FN],[FP,TN]]: \n {cm}\n")
print(f"Accuracy= {Accuracy}")
print(f"Recall= {Recall}")
print(f"Precision= {Precision}")
print(f"F1= {F1}")

Class Counts [0, 1]:
 [460 454]

Prior Probabilities [0, 1]:
 [0.50328228 0.49671772]

Confusion Matrix[[TP,FN],[FP,TN]]: 
 [[168  31]
 [  7 186]]

Accuracy= 0.9030612244897959
Recall= 0.8442211055276382
Precision= 0.96
F1= 0.8983957219251336
